In [15]:
# WRITE YOUR PROMPT HERE

sentence = "Show most popular sofas first"

In [16]:
from urllib.parse import quote_plus
from pydantic import BaseModel


class Filters(BaseModel):
    """All units are in centimetres"""
    width_min: int | None = None
    width_max: int | None = None
    depth_min: int | None = None
    depth_max: int | None = None
    height_min: int | None = None
    height_max: int | None = None
    product_name: str | None = None

    first_cheapest: bool = False
    color: str | None = None

    @property
    def is_product_search(self):
        return self.product_name is not None

    def to_query_params(self):
        params = ""

        if self.product_name:
            params += f"query={quote_plus(self.product_name)}&"
        if self.width_min:
            params += f"width.min={self.width_min}&"
        if self.width_max:
            params += f"width.max={self.width_max}&"
        if self.depth_min:
            params += f"depth.min={self.depth_min}&"
        if self.depth_max:
            params += f"depth.max={self.depth_max}&"
        if self.height_min:
            params += f"height.min={self.height_min}&"
        if self.height_max:
            params += f"height.max={self.height_max}&"

        if self.first_cheapest:
            params += f"order=PRICE_ASC&"

        if self.color:
            params += f"color={self.color}&"

        return params

In [17]:
import os
from openai import OpenAI

from dotenv import load_dotenv


load_dotenv()


client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_KEY"),
)


completion = client.beta.chat.completions.parse(
    model="google/gemini-flash-1.5",
    messages=[
        {
            "role": "system",
            "content": "Extract parameters from the users sentence. Convert units to cm. integers and put into output. "
                       "Leave fields as None if not specified. If user includes some product name, brand also include category in product_name"
                       "Like if it asks 'Give me sofas names JENNY', then product_name must be 'sofas JENNY'"
        },
        {"role": "user", "content": sentence},
    ],
    response_format=Filters,
)


filters = completion.choices[0].message.parsed


In [ ]:
print(completion.choices)
print(filters.to_query_params())

In [19]:

import json
from app.constants import HEADERS


limit = 10
offset = 0


category_to_id = {
    "sofa-couch": "156318",
    "kueche-moebel": "156182",
    "innenleuchte": "177561"
}

variables = {
  # "urlParams": filters.to_query_params(),
  "urlParams": "query=sofas+JENNY&",
  "locale": "de_DE",
  "first": limit,
  "offset": offset,
  "format": "WEBP",
}

if not filters.is_product_search:
    variables["id"] = category_to_id["sofa-couch"]

if filters.is_product_search:
    variables["query"] = filters.product_name

CATEGORY_SEARCH_HASH = "80021581f9cced0ece4065bd9c99d50a4e8cdfe749ab57cea87ef0a549bfb7b7"
PRODUCT_SEARCH_HASH = "49a969c017878e1a0bf0c6db3b8cc470fe096fb20dbe8dbba75130de498bfb94"

extensions = {
  "persistedQuery": {
    "version": 1,
    "sha256Hash": PRODUCT_SEARCH_HASH if filters.is_product_search else CATEGORY_SEARCH_HASH,
  }
}

compact_variables = json.dumps(variables, separators=(',', ':'))
compact_extensions = json.dumps(extensions, separators=(',', ':'))


import requests
from urllib.parse import quote

BASE_URL = "https://www.home24.de"
url = f"{BASE_URL}/graphql?extensions={quote(compact_extensions)}&variables={quote(compact_variables)}"


response = requests.get(url,headers=HEADERS,)

print(response.status_code)



200


In [20]:
print(variables)
print(extensions)

{'urlParams': 'query=sofas+JENNY&', 'locale': 'de_DE', 'first': 10, 'offset': 0, 'format': 'WEBP', 'query': 'sofas'}
{'persistedQuery': {'version': 1, 'sha256Hash': '49a969c017878e1a0bf0c6db3b8cc470fe096fb20dbe8dbba75130de498bfb94'}}


In [25]:
from pydantic import BaseModel


class Dimensions(BaseModel):
    width: int
    height: int
    depth: int


class Product(BaseModel):
    title: str
    price: float
    image_url: str
    dimensions: Dimensions | None = None
    url: str
    brand: str
    rating: float

products = []

def extract_list(data):
    if filters.is_product_search:
        return data["data"]["categories"]["articles"]
    else:
        return data["data"]["categories"][0]["categoryArticles"]["articles"]

json_data = response.json()
for product in extract_list(json_data):
    product_obj = Product(
        title=product["name"],
        price=product["prices"]["regular"]["value"],
        image_url=product["images"][0]["path"],
        url=BASE_URL + "/" + product["url"],
        brand=product["brand"]["name"],
        rating=product["ratings"]["average"]
    )
    products.append(product_obj)


In [26]:
import pandas as pd

df = pd.DataFrame([p.__dict__ for p in products])
df

,title,price,image_url,dimensions,url,brand,rating
0,Ecksofa Torezio Mini,84900.0,https://cdn1.home24.net/images/media/catalog/p...,None,https://www.home24.de/produkt/ecksofa-torezio-...,MIRJAN24,4.7
1,Ecksofa Karl Cord,47900.0,https://cdn1.home24.net/images/media/catalog/p...,None,https://www.home24.de/produkt/ecksofa-karl-cor...,MIRJAN24,4.0
2,Ecksofa COVENTRY mit Schlaffunktion,59999.0,https://cdn1.home24.net/images/media/catalog/p...,None,https://www.home24.de/produkt/ecksofa-coventry...,Mørteens,4.0
3,Ecksofa POTARO,52500.0,https://cdn1.home24.net/images/media/catalog/p...,None,https://www.home24.de/produkt/ecksofa-potaro-w...,MASSENO,4.4
4,Big Sofa MADELINE,199900.0,https://cdn1.home24.net/images/media/catalog/p...,None,https://www.home24.de/produkt/big-sofa-madelin...,KAWOLA,5.0
5,Ecksofa HUDSON 3-Sitzer mit Longchair,169999.0,https://cdn1.home24.net/images/media/catalog/p...,None,https://www.home24.de/produkt/ecksofa-hudson-3...,Studio Copenhagen,4.6
6,Ecksofa FARESE NEW,94999.0,https://cdn1.home24.net/images/media/catalog/p...,None,https://www.home24.de/produkt/ecksofa-farese-n...,Selsey,4.6
7,2-Sitzer Sofa HUDSON,89999.0,https://cdn1.home24.net/images/media/catalog/p...,None,https://www.home24.de/produkt/2-sitzer-sofa-hu...,Studio Copenhagen,4.6
8,Ecksofa Corintia mit Ottomane,179999.0,https://cdn1.home24.net/images/media/catalog/p...,None,https://www.home24.de/produkt/ecksofa-corintia...,loftscape,5.0
9,Sofa 3-Sitzer MADELINE,176900.0,https://cdn1.home24.net/images/media/catalog/p...,None,https://www.home24.de/produkt/sofa-3-sitzer-ma...,KAWOLA,5.0


In [ ]:
from pprint import pprint

pprint(json_data)